# Attempting to implement an RNN for single word text generation

Following the tutorial at [this link](https://www.tensorflow.org/text/tutorials/text_generation#train_the_model).

In [3]:
try:
    print("Importing packages")
    import tensorflow as tf
    import pandas as pd

    import numpy as np
    import os
    import time
    print("Done")
except Exception as e:
    print("Error in imports", e)

Importing packages
Done


In [ ]:
# Step 1: read in and analyze the text data

First, we will read in the data and see what we are looking at. 

In [ ]:
og_passwords = []
text = open("rockyou.txt", "rb").read().decode(encoding='utf-8', errors='ignore')
file = open("rockyou.txt", "r", encoding="utf-8", errors='ignore')
temp_lines = file.readlines()
file.close()
for l in temp_lines:
    l = l.strip()
    l = l.replace("\n", "")
    og_passwords.append(l)
print(og_passwords[:5]) # taking a look at the first five passwords,,, these will be in a list format

# Step 2: Vectorize the data (make the words into numbers)
We will use the tensorflow methods to assist with this. 

In [ ]:
vocab = sorted(set(text))
print(len(vocab))

There are 710 unique characters in our set of data. I am again following the tutorial from tensorflow, but this makes sense to me. Now we need to make a mapping from the characters to a numerical representation so that the models work correctly. 

# Examples from the tutorial
The following section is an example of how we can vectorize things. I think we would want to do this later. 

## Helper Methods (converting to a

In [ ]:
#example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

# Continuing with our project


There are 710 unique characters in our set of data. I am again following the tutorial from tensorflow, but this makes sense to me. Now we need to make a mapping from the characters to a numerical representation so that the models work correctly. 

# Examples from the tutorial
The following section is an example of how we can vectorize things. I think we would want to do this later. 

## Helper Methods (converting to a

In [33]:
#example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [35]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

# Continuing with our project
